In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedModel, PreTrainedTokenizer
# import torch

import numpy as np

/Users/rqiu/Developer/chitchat/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "databricks/dolly-v1-6b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [5]:
model

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50403, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

## Prompt and response

In [6]:
PROMPT_FORMAT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""

def generate_response(instruction: str, *, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, 
                      do_sample: bool = True, max_new_tokens: int = 256, top_p: float = 0.92, top_k: int = 0, **kwargs) -> str:
    input_ids = tokenizer(PROMPT_FORMAT.format(instruction=instruction), return_tensors="pt").input_ids.to("cpu")

    # each of these is encoded to a single token
    response_key_token_id = tokenizer.encode("### Response:")[0]
    end_key_token_id = tokenizer.encode("### End")[0]

    gen_tokens = model.generate(input_ids, pad_token_id=tokenizer.pad_token_id, eos_token_id=end_key_token_id,
                                do_sample=do_sample, max_new_tokens=max_new_tokens, top_p=top_p, top_k=top_k, **kwargs)[0].cpu()

    # find where the response begins
    response_positions = np.where(gen_tokens == response_key_token_id)[0]

    if len(response_positions) >= 0:
        response_pos = response_positions[0]
        
        # find where the response ends
        end_pos = None
        end_positions = np.where(gen_tokens == end_key_token_id)[0]
        if len(end_positions) > 0:
            end_pos = end_positions[0]

        return tokenizer.decode(gen_tokens[response_pos + 1 : end_pos]).strip()

    return None

In [7]:
# Sample similar to: "Excited to announce the release of Dolly, a powerful new language model from Databricks! #AI #Databricks"
generate_response("Write a tweet announcing Dolly, a large language model from Databricks.", model=model, tokenizer=tokenizer)

"You're going to love @Databricks's new language model, Dolly! Get it now and revolutionize text processing tasks! #AI #ML #Databricks"

In [8]:
generate_response("Who's the president of the United States and how old is he now?", model=model, tokenizer=tokenizer)

'The current President of the United States is Donald Trump, and he is 71 years old.'

In [9]:
generate_response("Write me a paragraph of the introduction to ChatGPT", model=model, tokenizer=tokenizer)

'Welcome to ChatGPT, a conversational AI chatbot for everyday use. ChatGPT is designed to help you connect with friends and family more easily by providing intelligent answers to your questions. ChatGPT uses natural language processing (NLP) to understand the context of your questions and provides relevant answers quickly.'

In [10]:
generate_response("Imitate Charles Dickens' writing style of A Tale of Two Cities to write a paragraph about social network's impact on teenagers today.", model=model, tokenizer=tokenizer)

'In a world where teenagers are connected to each other and to the world in unprecedented ways, the sheer number of social networks available has drastically changed the way teenagers interact and communicate. Social media has allowed teens to stay connected, even when their physical presence is not readily available. They can easily communicate with friends and family, share photos and videos, and connect with other people around the world. However, there are risks associated with the easy access to this level of technology. Teens have been exposed to inappropriate and sometimes dangerous content, they are increasingly spending time on these platforms without parental supervision, and they may be overly reliant on them. Parents should be mindful of the content that their children are exposed to online and help them understand the importance of self-control when using social media.'